In [29]:
import pandas as pd
from langchain_groq import ChatGroq
from decouple import AutoConfig
from langchain_community.document_loaders import WebBaseLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

In [30]:
config = AutoConfig(search_path="../.env")

GROQ_API_KEY = config("GROQ_API_KEY")
LANGCHAIN_TRACING_V2 = config("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = config("LANGCHAIN_ENDPOINT")
LANGCHAIN_PROJECT = config("LANGCHAIN_PROJECT")
lANGCHAIN_API_KEY = config("LANGCHAIN_API_KEY")

In [31]:
job_url = input("Paste the URL here: ")
print(f"{job_url=}")

job_url='https://www.amazon.jobs/en/jobs/2853585/senior-software-development-engineer'


In [32]:
web_data = WebBaseLoader(web_path=job_url).load()
llm = ChatGroq(model="llama-3.1-70b-versatile", api_key=GROQ_API_KEY, temperature=0.5)
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
json_parser = JsonOutputParser()
str_parser = StrOutputParser()

In [33]:
scrape_prompt = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        THe Scraped text is from the career page of a website.
        your job is to extract the job postings and return them in JSON format containing
        the following keys: `role`, `experience`, `skills` and `description`.
        Return only the valid JSON.
        ### VALID JSON (NO PREAMBLE):
    """
)

email_prompt = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    ### INSTRUCTION:
    You are Madhu, a business development executive at Mocha. 
    Mocha is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
    process optimization, cost reduction, and heightened overall efficiency. 
    Your job is to write a cold email to the client regarding the job mentioned above describing 
    the capability of Mocha in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Mocha's portfolio: {link_list}
    Remember you are Madhu, BDE at Mocha. 
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)

In [34]:
scrape_chain = scrape_prompt | llm | json_parser
email_chain = email_prompt | llm | str_parser

In [35]:
page_data = web_data.pop().page_content
print(scrape_prompt.format(**{"page_data":page_data}))


        ### SCRAPED TEXT FROM WEBSITE:
        Senior Software Development Engineer - Job ID: 2853585 | Amazon.jobs
Skip to main contentHomeTeamsLocationsJob categoriesMy careerMy applicationsMy profileAccount securitySettingsSign outResourcesDisability accommodationsBenefitsDiversity, equity, and inclusionInterview tipsLeadership principlesWorking at AmazonFAQ×Senior Software Development EngineerJob ID: 2853585 | ADCI HYD 13 SEZApply nowDESCRIPTIONWould you like to work on one of the world's largest transactional distributed systems? How about working with customers and peers from the entire range of Amazon's business on cool new features? Whether you're passionate about building highly scalable and reliable systems or a software developer who likes to solve business problems, Amazon Tax Platform Services is the place for you.We are responsible for the tax platform, providing the core services that calculate taxes (sales tax and VAT) for all Amazon sales, physical and digital, global

In [36]:
response = scrape_chain.invoke(input={"page_data": page_data})
print(response)

{'role': 'Senior Software Development Engineer', 'experience': '5+ years of non-internship professional software development experience', 'skills': ['programming with at least one software programming language experience', 'leading design or architecture (design patterns, reliability and scaling) of new and existing systems experience', 'Java, Linux and Oracle-based environment'], 'description': 'We are looking for software engineers who thrive on complex problems and relish the challenge of operating complex and mission critical systems under extreme loads. Our systems manage hundreds of millions of records, and respond to millions of service requests per minute.'}


In [37]:
df = pd.read_csv("../resources/my_portfolio.csv")
display(df)

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [38]:
csv_documents = []
for _, row in df.iterrows():
    csv_documents.append(Document(page_content=row["Techstack"], metadata={"source": row["Links"]}))

In [39]:
vector_store = Chroma(collection_name="portfolio_collection", embedding_function=embeddings)
vector_store.add_documents(csv_documents)

['0456788d-86cf-40e0-8662-30d3effa54b3',
 '48d21ac8-0937-4ba5-9030-50dfb5b6f823',
 '5292d8df-e3db-4d52-ba9c-3272a56d234a',
 '8d9c4de7-63e9-4c5b-bb3d-f20ea752f4a3',
 'decae6b8-bddb-401e-ac60-194a54364a6c',
 'bb69e771-3449-407c-baa1-65700eded39d',
 '5f696de1-00aa-4067-8f95-4365c5cf8b3e',
 '62873023-d186-427f-992d-00ec7a954daa',
 '32ce3b4b-5828-439f-8057-6636b10d95e2',
 '2ad23784-3dd0-4590-aa6d-e57e8ca84e34',
 '0533a1c1-777d-4921-b464-ea70f4ab47ae',
 'fb0f9825-df13-4ab2-8994-02bc5dea314d',
 '2e948f22-c78e-4517-9c6f-452a572819a8',
 '94761e3b-e412-4ec7-8f3e-def48da60325',
 '5dee100b-2f31-4bf3-95b1-d3c527c1e9bd',
 'd2dc7b1f-f8f0-4917-91fa-992c308ba22f',
 'aa494a12-9b4b-40ba-a877-a0f314be95a3',
 'f30a4c3e-824f-4e50-aec5-037abe64cd16',
 '69d77c64-a211-429a-a83b-985e0e361ce9',
 '8e7b589a-8e9c-4f36-b7d2-5354b96beaf8']

In [40]:
response['skills']

['programming with at least one software programming language experience',
 'leading design or architecture (design patterns, reliability and scaling) of new and existing systems experience',
 'Java, Linux and Oracle-based environment']

In [41]:
relevant_portfolios = []
for skill in response.get('skills'):
    relevant_portfolios.append(vector_store.similarity_search(skill, k=1)[0].metadata['source'])
relevant_portfolios

['https://example.com/ml-python-portfolio',
 'https://example.com/android-portfolio',
 'https://example.com/java-portfolio']

In [42]:
print(email_prompt.format(**{"job_description":str(response), "link_list":set(relevant_portfolios)}))


    ### JOB DESCRIPTION:
    {'role': 'Senior Software Development Engineer', 'experience': '5+ years of non-internship professional software development experience', 'skills': ['programming with at least one software programming language experience', 'leading design or architecture (design patterns, reliability and scaling) of new and existing systems experience', 'Java, Linux and Oracle-based environment'], 'description': 'We are looking for software engineers who thrive on complex problems and relish the challenge of operating complex and mission critical systems under extreme loads. Our systems manage hundreds of millions of records, and respond to millions of service requests per minute.'}
    ### INSTRUCTION:
    You are Madhu, a business development executive at Mocha. 
    Mocha is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterpr

In [43]:
response = email_chain.invoke({"job_description":str(response), "link_list":set(relevant_portfolios)})
print(response)

Subject: Expert Software Development Solutions for Complex Systems

Dear Hiring Manager,

I came across the job description for a Senior Software Development Engineer at your esteemed organization, and I am excited to introduce Mocha, an AI & Software Consulting company that can help you address your software development needs.

Our team of experts has extensive experience in designing and developing complex systems that manage large volumes of data and respond to high service requests. We understand the challenges of operating mission-critical systems under extreme loads and have a proven track record of delivering scalable and reliable solutions.

At Mocha, we have a strong proficiency in Java, Linux, and Oracle-based environments, which aligns perfectly with your requirements. Our team has experience in leading design or architecture of new and existing systems, ensuring that our solutions are efficient, reliable, and scalable.

I would like to highlight a few examples of our work i